In [ ]:
import imageio
import os

def create_gif_from_vtk(params):
    out_to_vtk = !ls -v1 {params.show_file}*vtk
    print(out_to_vtk)

    filenames = []

    for i, file in enumerate(out_to_vtk):
        p = pv.Plotter(notebook=True)

        mesh = pv.read(file)

        warped_mesh = mesh

        if params.warp_field_scalar:
            warped_mesh = mesh.warp_by_scalar(scalars=params.warp_field_scalar, factor=params.warp_factor)
        
        if params.warp_field_vector:
            warped_mesh = mesh.warp_by_vector(vectors=params.warp_field_vector, factor=params.warp_factor)

        if params.show_edges:
            warped_mesh=warped_mesh.shrink(0.95)

        if params.show_field in warped_mesh.point_data:
            data_location = warped_mesh.point_data
        elif params.show_field in warped_mesh.cell_data:
            data_location = warped_mesh.cell_data
        else:
            raise KeyError(f"Field '{params.show_field}' not found in point or cell data.")

        if (params.field_part < 0):
            scalars = data_location[params.show_field]
        elif params.field_part == 10: # plot gradient
            warped_mesh = warped_mesh.compute_derivative(scalars=params.show_field, preference='point')
            scalars = warped_mesh.point_data['gradient']
        else:
            scalars = warped_mesh.point_data[params.show_field][:,params.field_part]

        if params.show_ori_shape:
            p.add_mesh(mesh, component=None, smooth_shading=True, opacity=0.5, color='gray')

        scalar_bar_title = params.show_field if params.show_field else "Scalar Field"
             # Define the arguments for the scalar bar
        scalar_bar_args = {
            # 'n_labels': 8,
            'position_x': 0.2,
            'title': scalar_bar_title,

        }

        p.add_mesh(warped_mesh, scalars=scalars, component=None, smooth_shading=False, cmap=params.p_cmap, clim = params.clim, show_scalar_bar=params.show_scalar_bar, opacity=0.9, scalar_bar_args=scalar_bar_args)
    
        p.camera_position = params.camera_position
        
        p.enable_parallel_projection()
        p.enable_image_style()
        
        # Save the plot as an image with reduced resolution
        filename = params.gif_name+f"_{i}.png"
        p.screenshot(filename, window_size=[800, 600])  # Adjust the window size as needed
        
        # Append the filename to the list
        filenames.append(filename)

    # Create a gif from the image files with a 0.2 second delay between frames
    with imageio.get_writer(params.gif_name+'.gif', mode='I', duration=1) as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

    with imageio.get_writer(params.gif_name+'.mp4', fps=1) as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)
            
    # Optionally, delete the individual frame images
    for filename in filenames:
        os.remove(filename)